# Introduction to neural nets

## 2. Multi-layer perceptron
<figure align="middle">
    <img src="images/perceptron_2layers.png" title="2-layer perceptron" width="400px">
    <figcaption>Fig 1. Multi-layer perceptron with 1 hidden layer.</figcaption>
</figure>

Implementation of a 2-layer perceptron: input layer plus hidden layer plus output.

In [18]:
import numpy as np

class MLPerceptron:
    """ A Multi-Layer Perceptron"""

    def __init__(self, inputs, targets, nhidden, beta=1, momentum=0.9, outtype='logistic'):
        """ Constructor """
        # Set up network size
        self.nin = np.shape(inputs)[1]
        self.nout = np.shape(targets)[1]
        self.ndata = np.shape(inputs)[0]
        self.nhidden = nhidden

        self.beta = beta
        self.momentum = momentum
        self.outtype = outtype

        # Initialise network
        self.weights1 = (np.random.rand(self.nin + 1, self.nhidden) - 0.5) * 2 / np.sqrt(self.nin)
        self.weights2 = (np.random.rand(self.nhidden + 1, self.nout) - 0.5) * 2 / np.sqrt(self.nhidden)



    def train(self, inputs, targets, eta, niterations):
        """ Train the thing """
        # Add the bias node
        inputs_with_bias = np.concatenate((inputs, -np.ones((self.ndata, 1))), axis=1)
        change = range(self.ndata)

        updatew1 = np.zeros((np.shape(self.weights1)))
        updatew2 = np.zeros((np.shape(self.weights2)))

        current_iteration = 0
        for n in range(niterations):
            current_iteration += 1
            self.outputs = self.forward(inputs_with_bias)

            error = 0.5 * np.sum((self.outputs - targets) ** 2)
            
            if (np.mod(n, 50) == 0):
                print("Iteration: ", n+1, " Error: ", error)

            
            # Compute delta between targets and outputs
            # Different types of output neurons
            if self.outtype == 'linear':
                deltao = (self.outputs - targets) / self.ndata
            elif self.outtype == 'logistic':
                deltao = self.beta * (self.outputs - targets) * self.outputs * (1.0 - self.outputs)
            elif self.outtype == 'softmax':
                deltao = (self.outputs - targets) * (self.outputs * (-self.outputs) + self.outputs) / self.ndata
            else:
                print("error")

            
            # Compute delta for each weight from input to hidden
            deltah = self.hidden * self.beta * (1.0 - self.hidden) * (np.dot(deltao, np.transpose(self.weights2)))

            # backpropagation of error
            updatew1 = eta * (np.dot(np.transpose(inputs_with_bias), deltah[:, :-1])) + self.momentum * updatew1
            updatew2 = eta * (np.dot(np.transpose(self.hidden), deltao)) + self.momentum * updatew2
            self.weights1 -= updatew1
            self.weights2 -= updatew2

            cm, accuracy = self.confusion_matrix(inputs, targets)
            if accuracy == 1.0:
                break
        print("Total {} iterations".format(current_iteration))

    def forward(self, inputs):
        """ Run the network forward """

        self.hidden = np.dot(inputs, self.weights1);
        self.hidden = 1.0 / (1.0 + np.exp(-self.beta * self.hidden))
        self.hidden = np.concatenate((self.hidden, -np.ones((np.shape(inputs)[0], 1))), axis=1)

        outputs = np.dot(self.hidden, self.weights2);

        # Different types of output neurons
        if self.outtype == 'linear':
            return outputs
        elif self.outtype == 'logistic':
            return 1.0 / (1.0 + np.exp(-self.beta * outputs))
        elif self.outtype == 'softmax':
            normalisers = np.sum(np.exp(outputs), axis=1) * np.ones((1, np.shape(outputs)[0]))
            return np.transpose(np.transpose(np.exp(outputs)) / normalisers)
        else:
            print("error")

    def confusion_matrix(self, inputs, targets):
        """Confusion matrix"""

        # Add the inputs that match the bias node
        inputs = np.concatenate((inputs, -np.ones((np.shape(inputs)[0], 1))), axis=1)
        outputs = self.forward(inputs)

        nclasses = np.shape(targets)[1]

        if nclasses == 1:
            nclasses = 2
            outputs = np.where(outputs > 0.5, 1, 0)
        else:
            # 1-of-N encoding
            outputs = np.argmax(outputs, 1)
            targets = np.argmax(targets, 1)

        cm = np.zeros((nclasses, nclasses))
        for i in range(nclasses):
            for j in range(nclasses):
                cm[i, j] = np.sum(np.where(outputs == i, 1, 0) * np.where(targets == j, 1, 0))


        return cm, np.trace(cm) / np.sum(cm)

## 2.1. Learning AND
This still learns a linear separator

In [5]:
anddata = np.array([[0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 1]])

p = MLPerceptron(anddata[:, 0:2], anddata[:, 2:3], 2)
p.train(anddata[:, 0:2], anddata[:, 2:3], 0.25, 1001)

print()
cm, accuracy = p.confusion_matrix(anddata[:, 0:2], anddata[:, 2:3])
print("Confusion matrix:")
print(cm)
print("Accuracy: ", accuracy)

Iteration:  1  Error:  0.5898738819231876
Iteration:  11  Error:  0.4047180256129999
Iteration:  21  Error:  0.3966217145443324
Iteration:  31  Error:  0.35131811242340544
Iteration:  41  Error:  0.32573547634315564
Iteration:  51  Error:  0.2914836545540826
Iteration:  61  Error:  0.24301118621986115

Confusion matrix:
[[3. 0.]
 [0. 1.]]
Accuracy:  1.0


## 2.2. Learning XOR

In [6]:
xordata = np.array([[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0]])
q = MLPerceptron(xordata[:, 0:2], xordata[:, 2:3], 2, outtype='logistic')
q.train(xordata[:, 0:2], xordata[:, 2:3], 0.25, 5001)

print()
cm, accuracy = q.confusion_matrix(xordata[:, 0:2], xordata[:, 2:3])
print("Confusion matrix:")
print(cm)
print("Accuracy: ",accuracy)

Iteration:  1  Error:  0.6089216486579121
Iteration:  11  Error:  0.5375813545851306
Iteration:  21  Error:  0.5127565045601278
Iteration:  31  Error:  0.5042507648661128
Iteration:  41  Error:  0.5013152211453158
Iteration:  51  Error:  0.5002822468580538
Iteration:  61  Error:  0.4998977958475319
Iteration:  71  Error:  0.4997305882462564
Iteration:  81  Error:  0.49963145843309786
Iteration:  91  Error:  0.4995476307469885
Iteration:  101  Error:  0.4994587301597828
Iteration:  111  Error:  0.4993547162094239
Iteration:  121  Error:  0.4992277253156844
Iteration:  131  Error:  0.49906878039226776
Iteration:  141  Error:  0.4988653810432664
Iteration:  151  Error:  0.4985990169171213
Iteration:  161  Error:  0.498241006310948
Iteration:  171  Error:  0.49774566419639027
Iteration:  181  Error:  0.4970378555402127
Iteration:  191  Error:  0.4959907577641733
Iteration:  201  Error:  0.4943857273784021
Iteration:  211  Error:  0.49184303857522693
Iteration:  221  Error:  0.4877105468267

<figure align="middle">
    <img src="images/mlp_XOR.png" title="Separated XOR by hyperplane" width="400px">
    <figcaption>Fig 2. Separated the linearly non-separable XOR by a hyperplane.</figcaption>
</figure>

## 2.3. Breast cancer diagnosis

The simple 2-layer perceptron implemented above is quite powerful.
We are going to use it to classify mammography results into malignant and non-malignant. 
The dataset is from [here](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)).

In [12]:
data_file = "wdbc.csv"

In [13]:
import csv

def csv_to_array(file_name):
    np_a = None
    with open(file_name) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for row in readCSV:
            class_label = 0
            if row[0].strip()=='M':
                class_label = 1
            num_arr = [float(x) for x in row[1:]] + [class_label]

            if np_a is None:
                np_a = np.array(num_arr)
            else:
                np_a = np.c_[np_a, num_arr]

    np_a = np.transpose(np_a)

    return np_a

In [14]:
bc_data = csv_to_array(data_file)
n_rows = np.shape(bc_data)[0]
n_cols = np.shape(bc_data)[1]
print("Dataset contains",n_rows,"rows and", n_cols,"columns")

# keep target attribute 0 or 1
target_a = bc_data[:,n_cols-1:n_cols ]

# normalize data in each column by subtracting mean and dividing by variance
bc_data = bc_data[:, 0:n_cols-1]
bc_data = (bc_data - bc_data.mean(axis=0)) / bc_data.var(axis=0)

bc_data = np.c_[bc_data,target_a]

Dataset contains 569 rows and 31 columns


Train the network:

In [21]:
p = MLPerceptron(bc_data[:, 0:n_cols-1], bc_data[:, n_cols-1:n_cols], 2, outtype="linear")
p.train(bc_data[:, 0:n_cols-1], bc_data[:, n_cols-1:n_cols], 0.0005, 1000001)

Iteration:  1  Error:  68.06564309221797
Iteration:  51  Error:  30.220059861379
Iteration:  101  Error:  19.869147160387723
Iteration:  151  Error:  17.726110023460162
Iteration:  201  Error:  31.392199804033748
Iteration:  251  Error:  14.786376237321424
Iteration:  301  Error:  21.505586193376754
Iteration:  351  Error:  16.713551199486435
Iteration:  401  Error:  16.60518165001443
Iteration: 

<ipython-input-18-223a6f41ca85>:74: RuntimeWarning: overflow encountered in exp
  self.hidden = 1.0 / (1.0 + np.exp(-self.beta * self.hidden))


 451  Error:  16.834141062455267
Iteration:  501  Error:  12.146460836910176
Iteration:  551  Error:  10.636951881619026
Iteration:  601  Error:  17.827855586639636
Iteration:  651  Error:  16.42095059366858
Iteration:  701  Error:  12.060330807373195
Iteration:  751  Error:  11.401043573068248
Iteration:  801  Error:  10.923943897301442
Iteration:  851  Error:  9.574987270102255
Iteration:  901  Error:  9.41261644172014
Iteration:  951  Error:  9.358448924702435
Iteration:  1001  Error:  9.312638509194297
Iteration:  1051  Error:  9.288453390890943
Iteration:  1101  Error:  9.271591958591898
Iteration:  1151  Error:  9.258962514897084
Iteration:  1201  Error:  9.2486373782537
Iteration:  1251  Error:  9.552403207829304
Iteration:  1301  Error:  10.344546387464915
Iteration:  1351  Error:  10.298761614563098
Iteration:  1401  Error:  9.910691145815793
Iteration:  1451  Error:  9.701258252385752
Iteration:  1501  Error:  9.680709981835273
Iteration:  1551  Error:  9.677602057239458
Iter

Iteration:  9951  Error:  9.612503987778949
Iteration:  10001  Error:  9.612502073340165
Iteration:  10051  Error:  9.612500203085428
Iteration:  10101  Error:  9.612498375716068
Iteration:  10151  Error:  9.612496589974612
Iteration:  10201  Error:  9.612494844643361
Iteration:  10251  Error:  9.612493138543059
Iteration:  10301  Error:  9.612491470531562
Iteration:  10351  Error:  9.612489839502608
Iteration:  10401  Error:  9.612488244384592
Iteration:  10451  Error:  9.612486684139395
Iteration:  10501  Error:  9.612485157761274
Iteration:  10551  Error:  9.612483664275771
Iteration:  10601  Error:  9.61248220273867
Iteration:  10651  Error:  9.612480772234978
Iteration:  10701  Error:  9.612479371877969
Iteration:  10751  Error:  9.612478000808238
Iteration:  10801  Error:  9.612476658192804
Iteration:  10851  Error:  9.612475343224219
Iteration:  10901  Error:  9.612474055119756
Iteration:  10951  Error:  9.612472793120567
Iteration:  11001  Error:  9.612471556490917
Iteration:  

Iteration:  19551  Error:  9.60488372869835
Iteration:  19601  Error:  9.604879041487717
Iteration:  19651  Error:  9.60487448560232
Iteration:  19701  Error:  9.604870055147341
Iteration:  19751  Error:  9.604865744620028
Iteration:  19801  Error:  9.604861548873037
Iteration:  19851  Error:  9.604857463082222
Iteration:  19901  Error:  9.60485348271816
Iteration:  19951  Error:  9.604849603520979
Iteration:  20001  Error:  9.604845821477944
Iteration:  20051  Error:  9.604842132803562
Iteration:  20101  Error:  9.604838533921725
Iteration:  20151  Error:  9.604835021449748
Iteration:  20201  Error:  9.604831592183988
Iteration:  20251  Error:  9.604828243086924
Iteration:  20301  Error:  9.604824971275406
Iteration:  20351  Error:  9.604821774010055
Iteration:  20401  Error:  9.604818648685606
Iteration:  20451  Error:  9.604815592822106
Iteration:  20501  Error:  9.604812604056868
Iteration:  20551  Error:  9.60480968013712
Iteration:  20601  Error:  9.604806818913241
Iteration:  20

...

Iteration:  955201  Error:  9.255225854101793
Iteration:  955251  Error:  9.255225449260951
Iteration:  955301  Error:  9.25522504698639
Iteration:  955351  Error:  9.25522464725658
Iteration:  955401  Error:  9.255224250050277
Iteration:  955451  Error:  9.255223855346522
Iteration:  955501  Error:  9.255223463124619
Iteration:  955551  Error:  9.255223073364139
Iteration:  955601  Error:  9.255222686044911
Iteration:  955651  Error:  9.255222301147011
Iteration:  955701  Error:  9.25522191865076
Iteration:  955751  Error:  9.255221538536713
Iteration:  955801  Error:  9.255221160785661
Iteration:  955851  Error:  9.255220785378611
Iteration:  955901  Error:  9.255220412296795
Iteration:  955951  Error:  9.255220041521653
Iteration:  956001  Error:  9.255219673034826
Iteration:  956051  Error:  9.25521930681817
Iteration:  956101  Error:  9.255218942853723
Iteration:  956151  Error:  9.255218581123717
Iteration:  956201  Error:  9.25521822161057
Iteration:  956251  Error:  9.255217864

Iteration:  964351  Error:  9.255179348717947
Iteration:  964401  Error:  9.255179182904604
Iteration:  964451  Error:  9.255179017513282
Iteration:  964501  Error:  9.255178852537338
Iteration:  964551  Error:  9.255178687970162
Iteration:  964601  Error:  9.25517852380515
Iteration:  964651  Error:  9.255178360035718
Iteration:  964701  Error:  9.2551781966553
Iteration:  964751  Error:  9.255178033657351
Iteration:  964801  Error:  9.25517787103533
Iteration:  964851  Error:  9.255177708782721
Iteration:  964901  Error:  9.255177546893009
Iteration:  964951  Error:  9.2551773853597
Iteration:  965001  Error:  9.255177224176297
Iteration:  965051  Error:  9.255177063336323
Iteration:  965101  Error:  9.2551769028333
Iteration:  965151  Error:  9.255176742660755
Iteration:  965201  Error:  9.25517658281223
Iteration:  965251  Error:  9.255176423281249
Iteration:  965301  Error:  9.255176264061362
Iteration:  965351  Error:  9.255176105146099
Iteration:  965401  Error:  9.2551759465289

Iteration:  973351  Error:  9.255143807385448
Iteration:  973401  Error:  9.255143125555831
Iteration:  973451  Error:  9.255142380193192
Iteration:  973501  Error:  9.255141556643261
Iteration:  973551  Error:  9.255140634693099
Iteration:  973601  Error:  9.25513958548136
Iteration:  973651  Error:  9.25513836603477
Iteration:  973701  Error:  9.255136908981086
Iteration:  973751  Error:  9.25513510162644
Iteration:  973801  Error:  9.255132739049882
Iteration:  973851  Error:  9.25512940466215
Iteration:  973901  Error:  9.255124107569127
Iteration:  973951  Error:  9.255113847700503
Iteration:  974001  Error:  9.25508453007895
Iteration:  974051  Error:  9.254786308939572
Iteration:  974101  Error:  9.110188953484398
Iteration:  974151  Error:  9.09340825583504
Iteration:  974201  Error:  9.091782212987894
Iteration:  974251  Error:  9.091182982052459
Iteration:  974301  Error:  9.090814288324621
Iteration:  974351  Error:  9.090558819607509
Iteration:  974401  Error:  9.0903732620

Iteration:  982401  Error:  9.059053260742013
Iteration:  982451  Error:  9.059052728389736
Iteration:  982501  Error:  9.059052205683635
Iteration:  982551  Error:  9.05905169228657
Iteration:  982601  Error:  9.059051187875255
Iteration:  982651  Error:  9.05905069213959
Iteration:  982701  Error:  9.05905020478206
Iteration:  982751  Error:  9.059049725517163
Iteration:  982801  Error:  9.059049254070846
Iteration:  982851  Error:  9.059048790180011
Iteration:  982901  Error:  9.05904833359199
Iteration:  982951  Error:  9.059047884064103
Iteration:  983001  Error:  9.059047441363203
Iteration:  983051  Error:  9.05904700526525
Iteration:  983101  Error:  9.059046575554907
Iteration:  983151  Error:  9.059046152025177
Iteration:  983201  Error:  9.059045734477019
Iteration:  983251  Error:  9.059045322719008
Iteration:  983301  Error:  9.059044916567007
Iteration:  983351  Error:  9.059044515843862
Iteration:  983401  Error:  9.059044120379095
Iteration:  983451  Error:  9.059043730

Iteration:  991451  Error:  9.059004665190935
Iteration:  991501  Error:  9.059004475216211
Iteration:  991551  Error:  9.059004285545225
Iteration:  991601  Error:  9.05900409617594
Iteration:  991651  Error:  9.059003907106344
Iteration:  991701  Error:  9.059003718334466
Iteration:  991751  Error:  9.059003529858366
Iteration:  991801  Error:  9.05900334167613
Iteration:  991851  Error:  9.059003153785888
Iteration:  991901  Error:  9.059002966185787
Iteration:  991951  Error:  9.05900277887401
Iteration:  992001  Error:  9.059002591848776
Iteration:  992051  Error:  9.059002405108323
Iteration:  992101  Error:  9.059002218650921
Iteration:  992151  Error:  9.05900203247487
Iteration:  992201  Error:  9.05900184657849
Iteration:  992251  Error:  9.059001660960131
Iteration:  992301  Error:  9.059001475618171
Iteration:  992351  Error:  9.059001290551013
Iteration:  992401  Error:  9.059001105757075
Iteration:  992451  Error:  9.059000921234812
Iteration:  992501  Error:  9.059000736

In [22]:
cm, accuracy = p.confusion_matrix(bc_data[:, 0:n_cols-1], bc_data[:, n_cols-1:n_cols])
print("Confusion matrix:")
print(cm)
print("Accuracy: ",accuracy)

Confusion matrix:
[[352.  17.]
 [  5. 195.]]
Accuracy:  0.961335676625659


<ipython-input-18-223a6f41ca85>:74: RuntimeWarning: overflow encountered in exp
  self.hidden = 1.0 / (1.0 + np.exp(-self.beta * self.hidden))


Copyright &copy; 2020 Marina Barsky. All rights reserved.